In [70]:
#import ml packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer



from sklearn.feature_extraction.text import CountVectorizer
import sklearn.model_selection as model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [71]:
#import dataset
train_df = pd.read_csv('../AskReddit Dataset/train.csv')
test_df = pd.read_csv('../AskReddit Dataset/test.csv')
train_df.head()

,qid,question_text,target
0,a3dee568776c08512c89,What is the role of Lua in Civ4?,0
1,bdb84f519e7b46e7b7bb,What are important chapters in Kannada for 10 ICSE 2018?,0
2,29c88db470e2eb5c97ad,Do musicians get royalties from YouTube?,0
3,3387d99bf2c3227ae8f1,What is the difference between Scaling Social Enterprises and Social Franchising?,0
4,e79fa5038f765d0f2e7e,Why do elevators go super slow right before the doors open?,0


In [72]:
# choose elements from df where target = 1
pd.set_option('display.max_colwidth', 100)
df_1 = train_df[train_df['target'] == 1]
df_1["question_text"]

16                                                     What stupid things do Indians do when in your country?
31                             Can I sue my parents for giving birth to me when I did not want them to do so?
32                          What are your views about sexual relationship between a widow mother and her son?
33        You became an atheist, and after 2 years you fall and break your back. You are left paralyzed fr...
90                                    Why aren't we protesting for government control instead of gun control?
                                                         ...                                                 
652967              What is a liberal's understanding of the difference between pollution and climate change?
653021    Do unattractive or average-looking men ever get a girlfriend who actually loves them or do they ...
653029                                                                   How can I grab my aunties boobs! :p?
653034    

In [73]:
# see value count order of target
train_df['target'].value_counts()

0    612656
1     40405
Name: target, dtype: int64

In [74]:
# percentage of troll questions in the dataset
train_df['target'].value_counts(normalize=True)

0    0.93813
1    0.06187
Name: target, dtype: float64

In [75]:
# create a preprocessing class
class Preprocessor:
    
    def __init__(self,df) -> None:
        self.df = df

    # convert all charecters to lower case
    def convertToLower(self):
        self.df['question_text'] = self.df['question_text'].apply(lambda x: x.lower())
        return self.df

    # remove stop words
    def removeStopWords(self):
        stop = stopwords.words('english')
        self.df['question_text'] = self.df['question_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
        return self.df


    # remove punctuation
    def removePunctuation(self):
        self.df['question_text'] = self.df['question_text'].str.replace('[^\w\s]','')
        return self.df

    # remove numbers
    def removeNumbers(self):
        self.df['question_text'] = self.df['question_text'].str.replace('[0-9]','')
        return self.df


    # stemmer algorithm
    def stemmer(self):
        stemmer = PorterStemmer()
        def stem_words(text):
            return " ".join([stemmer.stem(word) for word in text.split()])  
        self.df["question_text"] = self.df["question_text"].apply(lambda x: stem_words(x))
        return self.df


    # lemmatizing
    def lemmatize(self):
        from nltk.stem import WordNetLemmatizer
        lemmatizer = WordNetLemmatizer()
        def lemmatize_words(text):
            return " ".join([lemmatizer.lemmatize(word) for word in text.split()])  
        self.df["question_text"] = self.df["question_text"].apply(lambda x: lemmatize_words(x))
        return self.df

    # remove id and index columns
    def removeUnwantedCols(self,col):
        print(self.df.shape)
        self.df = self.df.drop(col,axis=1)
        return self.df
    
    def preprocess(self):
        # self.df = self.convertToLower()
        # self.df = self.removeStopWords()
        # self.df = self.removePunctuation()
        # self.df = self.removeNumbers()
        # self.df = self.stemmer()
        # self.df = self.lemmatize()
        self.df = self.removeUnwantedCols(['qid'])
        return self.df

In [76]:
preproccesor = Preprocessor(train_df)
preprocessed_df = preproccesor.preprocess()
preprocessed_df.head()

(653061, 3)


,question_text,target
0,What is the role of Lua in Civ4?,0
1,What are important chapters in Kannada for 10 ICSE 2018?,0
2,Do musicians get royalties from YouTube?,0
3,What is the difference between Scaling Social Enterprises and Social Franchising?,0
4,Why do elevators go super slow right before the doors open?,0


In [77]:
# get shape of preprocessed_df
preprocessed_df.shape

(653061, 2)

In [78]:
# create a get train and test data class
class TrainTestData:

    def __init__(self,trainDf,testDf) -> None:
        self.trainDf = trainDf
        self.testDf = testDf

    
    def get_X(self,minDocumentCount):
        vectorizer = CountVectorizer()
        self.trainData = vectorizer.fit_transform(self.trainDf['question_text'])
        print(self.trainData.shape)

        self.testData = vectorizer.transform(self.testDf['question_text'])
        print(self.testData.shape)
        self.X = self.trainData


    def get_Y(self):
        self.Y = self.trainDf['target']
        return self.Y

    def testTrainSplit(self):
        self.X_train, self.X_test, self.Y_train, self.Y_test = model_selection.train_test_split(self.X, self.Y, test_size=0.2, random_state=0)
        return self.X_train, self.X_test, self.Y_train, self.Y_test

    def get_X_test(self):
        return self.testData

In [79]:
testPreprocessor = Preprocessor(test_df)
preprocessed_test_df = testPreprocessor.preprocess()
preprocessed_test_df.head()

getTTData = TrainTestData(preprocessed_df,preprocessed_test_df)
X = getTTData.get_X(1)
y = getTTData.get_Y()
X_train,X_test,Y_train,Y_test = getTTData.testTrainSplit()

(653061, 2)
(653061, 135680)
(653061, 135680)


# Binary Classification using logistic regression

In [80]:
from sklearn.linear_model import LogisticRegression
lrModel = LogisticRegression(max_iter=1000)
lrModel.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [81]:
y_actual = Y_test
cv_preds = lrModel.predict(X_test)
print(cv_preds.shape)
from sklearn.metrics import fbeta_score
fbeta_score(y_actual, cv_preds, average='macro', beta=0.5)

(130613,)


0.7959226010052363

In [82]:
# linear svm classification
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(X_train, Y_train)


C:\Python39\lib\site-packages\sklearn\svm\_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [83]:
y_actual = Y_test
cv_preds = model.predict(X_test)

from sklearn.metrics import fbeta_score
fbeta_score(y_actual, cv_preds, average='macro', beta=0.5)

0.7818606859969338

# Binary classification using Multinomial NB

In [87]:
from sklearn.naive_bayes import MultinomialNB
mulNbClf = MultinomialNB().fit(X_train,Y_train)
mulNbPreds = mulNbClf.predict(X_test)
fbeta_score(Y_test, mulNbPreds, average='macro', beta=0.5)
np.unique(mulNbPreds, return_counts=True)

(array([0, 1], dtype=int64), array([130324,    289], dtype=int64))

# Creating a csv out of the test data for submission

In [84]:
preprocessed_test_df.shape

(653061, 1)

In [85]:
predictions = model.predict(getTTData.get_X_test())
predictions.shape

(653061,)

In [86]:
np.unique(predictions)

array([0, 1], dtype=int64)

In [87]:
# get number of unique counts of predictions
np.unique(predictions, return_counts=True)

(array([0, 1], dtype=int64), array([624772,  28289], dtype=int64))

In [88]:
# create a submission data frame from preprocessed_test_df and predictions
submission_df = pd.DataFrame({'qid': test_df['qid'], 'target': predictions})

In [89]:
submission_df.head()

,qid,target
0,0a824224322f0a36025f,0
1,28af14c4e4777ce1273e,1
2,6892a52c51103dd95044,0
3,badd9e8886d73fc1fe4e,0
4,4ef178f82a465e4804ae,0


In [90]:
submission_df['target'].value_counts()

0    624772
1     28289
Name: target, dtype: int64

In [91]:
# convert submission_df to admiral_general_aladdin_submission1.csv
submission_df.to_csv('admiral_general_aladdin_submission9_svc.csv', index=False)